In [1]:
## Project 5 - MLB Win/Loss Data w/ Pandas 
## Ben Rothman
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date
from datetime import timedelta
import csv
import os



DEBUG = False # toggle this var to turn on or off

In [2]:
## read the data from the files
print('program started ' + datetime.now().strftime("%H:%M:%S") + ' 🚩')

print('reading started ' + datetime.now().strftime("%H:%M:%S"))

# Variables
team_names = dict()

# support functions:
def read_data():
    """read each file in the 'data' diirectory to get the games data for every game played in the datafiles."""
    
    if DEBUG:
        # use dummy paths for a faster example
        paths = ['data/GL2014.TXT', 'data/GL2013.TXT', 'data/GL2012.TXT', 'data/GL2011.TXT', 'data/GL1896.TXT']
    else:
        # list all files in the data directory
        cd = os.getcwd()
        if cd.endswith('/data') == False:
            os.chdir('data')
        paths = [f for f in os.listdir('.')]
  
    # instantiate data to save all game data in
    data = pd.DataFrame(data=[])
    
    # read every item in 'paths'
    for path in paths:
        # read the current data file and only extract the columns we need into 'year_df'
        try:
            year_df = pd.read_csv(path, header = None, usecols=[0, 3, 6, 9, 10])
        except:
            print('could not read ' + path)

        # rename the columns in year_df to be human readable
        year_df = year_df.rename(columns = {year_df.columns[0]: "date", year_df.columns[1]: "away_team", year_df.columns[2]: "home_team", year_df.columns[3]: "away_score", year_df.columns[4]: "home_score"})

        # clean the data. convert the date into just the year and home_score + away_score into ints
        year_df['date'] = year_df.date.transform(lambda x: str(x)[0:4])
        year_df['home_score'] = year_df.home_score.transform(lambda x: int(x))
        year_df['away_score'] = year_df.away_score.transform(lambda x: int(x))

        # rename the 'date' column 'year' to make sense
        year_df = year_df.rename(columns={year_df.columns[0]: "year"})

        # add results to 'data'
        data = pd.concat([data, year_df], ignore_index=True, sort=False)
    return data

def read_team_names():
    '''Read the team name file to build a dictionary of team names along with their years'''
    try:
        with open('ids.txt', newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=',')
            for row in reader:
                team = row[0]
                city = row[2]
                nickname = row[3]
                start_date = int(row[4])
                end_date = int(row[5])
                for year in range(start_date, end_date):
                    team_names[str(year) + ' ' + team] = city + ' ' + nickname
    except:
        print(os.getcwd())

def lookup_team(year, team):
    '''Lookup a team name with the abbreviation and the year'''
    try:
        return team_names[str(year) + ' ' + team] + ' (' + team + ')'
    except:
        return team


#########################


# read team names
read_team_names()

# read data columns
all_games = read_data()

print('reading finished ' + datetime.now().strftime("%H:%M:%S"))

program started 09:51:31 🚩
reading started 09:51:31
could not read .DS_Store
could not read glfields.txt
reading finished 09:51:33


In [3]:
## Analysis - Part 1 - Great Team Performances ##
# Calculate the expected win/loss % for each team
# from the years 1871-2018. List the top 50 teams performances - measured
# by how many games the team outperformed (more wins) their expected win
# loss % in the given year. 

print('part 1 started at ' + datetime.now().strftime("%H:%M:%S"))

# find every recorded year
years = pd.unique(all_games['year'])

# variables
team_deviations = pd.DataFrame(columns=['Year', 'Team', 'Exp. Wins', 'Actual Wins', 'Deviation'])

# iterate through every year
for index, rowy in pd.DataFrame(years).iterrows():
    year = rowy[0]
    print('processing year ' + str(index + 1) + '/' + '148', end = "\r") # debug
    
    # find all teams that played during the current year
    teams = all_games[all_games['year'] == year]
    teams = teams['home_team'].unique()

    # iterate through every team that played that year to calculate their stats
    for i, rowt in pd.DataFrame(teams).iterrows():
        team = rowt[0]
        
        # get all home games + away games of 'team' for this year, then rename the resulting columns to be human readable
        home_games = all_games[(all_games['home_team'] == team) & (all_games['year'] == year)]
        home_games = home_games.rename(columns = {home_games.columns[0]: "year", home_games.columns[1]: "away_team", home_games.columns[2]: "home_team", home_games.columns[3]: "away_score", home_games.columns[4]: "home_score"})
        away_games = all_games[(all_games['away_team'] == team) & (all_games['year'] == year)]
        away_games = away_games.rename(columns = {away_games.columns[0]: "year", away_games.columns[1]: "away_team", away_games.columns[2]: "home_team", away_games.columns[3]: "away_score", away_games.columns[4]: "home_score"})
        
        # find how many games a team played in the current year by combining the lengths of home_games and away_games
        games_played = len(home_games) + len(away_games)

        # find how many games a team won and lost during the current year by combining home games won and away games won
        actual_wins = len(home_games.query('home_score > away_score')) + len(away_games.query('home_score < away_score'))
        actual_losses = len(home_games.query('home_score < away_score')) + len(away_games.query('home_score > away_score'))
        
        # get runs scored + runs allowed by 'team' during the current year
        scored = home_games['home_score'].sum() + away_games['away_score'].sum()
        allowed = home_games['away_score'].sum() + away_games['home_score'].sum()

        # calculate Expected Winning Percentage = Runs Scored2 / (Runs Scored2 + Runs Allowed2) for 'team' (unless data is missing)
        if scored != 0 and allowed != 0: # check if values are 0 or missing
            expected_wins = round((scored**2 / (scored**2 + allowed**2) * games_played), 1)
        else:
            expected_wins = actual_wins
            
        # calculate deviation for 'team' during the current year
        deviation = round(actual_wins - expected_wins, 1)
        
        # build the row for the current team + add the finished row of the current year's stats to 'team_deviations'
        row = pd.DataFrame([[year, lookup_team(year, team), expected_wins, actual_wins, deviation]], columns = ['Year', 'Team', 'Exp. Wins', 'Actual Wins', 'Deviation'])
        team_deviations = pd.concat([team_deviations, row], ignore_index = True)
        
print('reading finished at ' + datetime.now().strftime("%H:%M:%S"))    
# set part1 to a sorted version of 'team_deviations' by teams's highest outperformance of their deviation + get top 50
part1 = team_deviations.sort_values(by = ['Deviation'], ascending = False, ignore_index = True).head(50)

# dispaly part1 (positive deviation = outperformance)
display(part1)
print('part 1 finished at ' + datetime.now().strftime("%H:%M:%S"))

part 1 started at 09:51:33
reading finished at 09:53:58


,Year,Team,Exp. Wins,Actual Wins,Deviation
0,1905,Detroit Tigers (DET),64.4,79,14.6
1,2017,San Diego Padres (SDN),57.3,71,13.7
2,2016,Texas Rangers (TEX),81.9,95,13.1
3,2005,Arizona Diamondbacks (ARI),64.5,77,12.5
4,2018,Seattle Mariners (SEA),77.0,89,12.0
5,1972,New York Mets (NYN),71.0,83,12.0
6,1955,Kansas City Athletics (KC1),51.0,63,12.0
7,1943,Boston Braves (BSN),56.0,68,12.0
8,1984,New York Mets (NYN),78.1,90,11.9
9,1974,San Diego Padres (SDN),48.3,60,11.7


part 1 finished at 09:53:58


In [4]:
## Analysis - Part 2 - Historically Lucky and Unlucky Teams ##
# Calculate the average deviation (in games) between actual
# performance and expected performance across all years for
# each team, and list out the top 10 overperforming teams
# (actually won more games than expected), and top 10 underperforming
# teams (actually won less games than expected). Only include
# teams that played more than 10 years.

print('part 2 started at ' + datetime.now().strftime("%H:%M:%S"))

# instantiate empty dataframe for part 2
part2 = pd.DataFrame(columns=['Team', '# of Years', 'Avg. Wins', 'Exp. Wins', 'Avg. Deviation'])

# get the name of every team
teams = pd.unique(team_deviations['Team'])

# add the stats of every team to 'part2'
for i, row in pd.DataFrame(teams).iterrows():
    team = str(row[0])
    
    # get all of the deviations for every year the team played
    seasons = team_deviations[team_deviations['Team'] == team]
    
    if len(seasons) >= 10: # only use a team that has ten or more years
        number_of_years = len(seasons)
        avg_wins = seasons['Actual Wins'].mean()
        avg_expected = seasons['Exp. Wins'].mean()
        avg_deviation = seasons['Deviation'].mean()
#         # Team 	# of Years 	Avg. Wins 	Avg. Exp. Wins 	Avg. Deviation
        row = pd.DataFrame({'Team':team, '# of Years':number_of_years, 'Avg. Wins':round(avg_wins, 1), 'Exp. Wins':round(avg_expected, 1), 'Avg. Deviation':round(avg_deviation, 1)}, index=[1])
        part2 = pd.concat([part2, row], ignore_index = True)

# sort part 2 by Average Deviation over every season a team has played      
part2 = part2.sort_values(by = ['Avg. Deviation'], ascending = False)
# display(part2)

# get and display top 10 historically overperforming teams
overperforming = part2.head(10)
print('Overperforming:')
display(overperforming)

# get and display top ten historically underperforming teams
underperforming = part2.tail(10)
print('Underperforming:')
display(underperforming)

print('part 2 finished at ' + datetime.now().strftime("%H:%M:%S"))

part 2 started at 09:53:58
Overperforming:


,Team,# of Years,Avg. Wins,Exp. Wins,Avg. Deviation
5,Florida Marlins (FLO),18,75.7,74.5,1.2
8,San Diego Padres (SDN),50,73.5,72.4,1.1
41,Washington Senators (WS2),10,67.7,66.8,0.9
18,Anaheim Angels (ANA),22,85.9,85.0,0.9
40,Kansas City Athletics (KC1),12,63.9,63.1,0.8
38,California Angels (CAL),31,76.5,75.9,0.6
39,Montreal Expos (MON),35,76.8,76.4,0.4
27,Kansas City Royals (KCA),50,76.8,76.6,0.3
13,Seattle Mariners (SEA),42,75.0,74.8,0.2
34,Brooklyn Dodgers (BRO),67,77.4,77.2,0.2


Underperforming:


,Team,# of Years,Avg. Wins,Exp. Wins,Avg. Deviation
24,Chicago White Sox (CHA),118,78.1,78.8,-0.8
3,Chicago Cubs (CHN),143,77.1,78.1,-0.9
22,Pittsburgh Pirates (PIT),132,78.1,79.0,-1.0
25,New York Mets (NYN),57,76.5,77.6,-1.0
1,Washington Nationals (WAS),14,80.6,81.7,-1.1
19,Toronto Blue Jays (TOR),42,79.0,80.1,-1.1
16,Colorado Rockies (COL),26,75.5,76.8,-1.3
37,Milwaukee Braves (MLN),12,88.3,90.0,-1.7
31,New York Giants (NY1),74,81.9,84.1,-2.2
36,Cleveland Spiders (CL4),10,79.7,82.8,-3.1


part 2 finished at 09:53:58


In [5]:
## EXTRA CREDIT - Analysis - Part 3 - Home Field Advantage? ##
# Now let's see if home field advantage helps teams outperform
# their expected win/loss totals. Each game listed in each file
# has a home and away team listed. For this analysis, let's 
# compute the home team's actual win total (count how many times
# the home team won) against the home team's (all home teams!)
# expected total. This really amounts to recalculating win/loss
# expectations specifically for the generic "home team". Do this
# individually for each year between 1871-2018, and then calculate
# the average.

home_team_df = pd.DataFrame()

print('part 3 started at ' + datetime.now().strftime("%H:%M:%S"))

for year in years:
    year_games = all_games[all_games['year'] == year]

    # get runs scored by home teams for the current year
    home_runs_scored = year_games['home_score'].sum()
    
    # get runs scored by away teams for the current year
    away_runs_scored = year_games['away_score'].sum()
    
    # get home games played
    home_games_played, away_games_played = len(year_games), len(year_games)
    
    
    # get runs allowed by home teams for the current year
    home_runs_allowed = year_games['away_score'].sum()
    
    # get runs allowed by home teams for the current year
    away_runs_allowed = year_games['away_score'].sum()
    
    
    # get actual wins by home teams for the current year
    home_actual_wins = len(year_games.query('home_score > away_score'))
    
    # get actual wins by away teams for the current year
    away_actual_wins = len(year_games.query('home_score < away_score'))
    
    
    # calculate home expected wins for the current year Expected Winning Percentage = Runs Scored2 / (Runs Scored2 + Runs Allowed2) for 'team' (unless data is missing)
    home_expected_wins = round((home_runs_scored**2 / (home_runs_scored**2 + home_runs_allowed**2) * home_games_played), 1)
    
    # calculate away expected wins for the current year Expected Winning Percentage = Runs Scored2 / (Runs Scored2 + Runs Allowed2) for 'team' (unless data is missing)
    away_expected_wins = round((away_runs_scored**2 / (away_runs_scored**2 + away_runs_allowed**2) * away_games_played), 1)
    
    
    # calculate home deviation
    home_deviation = round(home_actual_wins - home_expected_wins, 1)
    
    # calculate away deviation
    away_deviation = round(away_actual_wins - away_expected_wins, 1)
    
    # Year 	Team 	# Actual Wins 	# Expected Wins 	Deviation
    home_row = pd.DataFrame({'Year':year, 'Team':'Home', 'Num Actual Wins':home_actual_wins, 'Num Expected Wins':home_expected_wins, 'Deviation':home_deviation}, index=[1])
    away_row = pd.DataFrame({'Year':year, 'Team':'Away', 'Num Actual Wins':away_actual_wins, 'Num Expected Wins':away_expected_wins, 'Deviation':away_deviation}, index=[1])
    home_team_df = pd.concat([home_team_df, home_row, away_row], ignore_index = True)

# show home team table
display(home_team_df)

# calculate the average deviations for only the home team
part3 = home_team_df[home_team_df['Team'] == 'Home']['Deviation'].mean()

# display the result
print('the average deviation for "The Home Team" is ' + str(round(part3, 1)) + ' games\n')
    
print('part 3 finished at ' + datetime.now().strftime("%H:%M:%S"))

print('program completed at ' + datetime.now().strftime("%H:%M:%S") + ' 🏁')


part 3 started at 09:53:58


,Year,Team,Num Actual Wins,Num Expected Wins,Deviation
0,2008,Home,1351,1279.0,72.0
1,2008,Away,1077,1214.0,-137.0
2,1918,Home,568,538.9,29.1
3,1918,Away,438,508.0,-70.0
4,1930,Home,704,655.4,48.6
...,...,...,...,...,...
291,1915,Away,843,932.0,-89.0
292,2011,Home,1276,1224.5,51.5
293,2011,Away,1153,1214.5,-61.5
294,2005,Home,1306,1255.2,50.8


the average deviation for "The Home Team" is 25.3 games

part 3 finished at 09:54:00
program completed at 09:54:00 🏁
